In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
plt.scatter(df['TotalVotes'], df['Positive%'])
plt.show()

df.drop(['NoVotes', 'YesVotes'], axis = 1, inplace = True)

correlation_beta = df[df['Template'] == 'BetaContentPage'].corr()
correlation_old = df[df['Template'] == 'AdviceguidePage'].corr()
correlation_benefits = df[df['Page'].str.startswith('/benefits/')].corr()
